# Notebook for Airconics examples 

IMPORTANT!: ipython notebook bug requires first code cells to be run manually (i.e. not with 'Run All' Cells). After this, all cells can be rerun with 'Run all below'. 

In [ ]:
import airconics
import airconics.AirCONICStools as act
# %gui qt4
# # Initialise the display
# from OCC.Display.SimpleGui import init_display
# display, start_display, add_menu, add_function_to_menu = init_display()

## Parameter Definitions

In [ ]:
Propulsion=1
EngineDia=2.9
FuselageScaling=[55.902, 55.902, 55.902]
NoseLengthRatio=0.182
TailLengthRatio=0.293
WingScaleFactor=44.56
WingChordFactor=1.0
Topology=1
SpanStation1=0.31
SpanStation2=0.625
EngineCtrBelowLE=0.3558
EngineCtrFwdOfLE=0.9837
Scarf_deg=3

# Derived Parameters
FuselageHeight = FuselageScaling[2]*0.105
FuselageLength = FuselageScaling[0]
FuselageWidth  = FuselageScaling[1]*0.106
WingApex = [0.1748*FuselageLength,0,-0.0523*FuselageHeight]
# Fin:
FinChordFact = 1.01
FinScaleFact = WingScaleFactor/2.032
# TailPlane
TPChordFact = 1.01
TPScaleFact = WingScaleFactor * 0.388


## Wing, Transonic Airliner

TODO: I tried making the lifting surface loft through airfoil sections as a solid loft, however the output stp file was not correct in Rhino (may be because TE and ends are not closed)

In [ ]:
# Import all example functional definitions for the Common Research Model (CRM) Wing:
from airconics.examples.wing_example_transonic_airliner import *

# Position of the apex of the wing
P = WingApex

# Class definition
NSeg = 10
ChordFactor = 1
ScaleFactor = 50

# Generate (surface building is done during construction of the class)
Wing = liftingsurface.LiftingSurface(P, mySweepAngleFunctionAirliner, 
                                        myDihedralFunctionAirliner, 
                                        myTwistFunctionAirliner, 
                                        myChordFunctionAirliner, 
                                        myAirfoilFunctionAirliner,
                                        SegmentNo=NSeg,
                                        ScaleFactor=WingScaleFactor,
                                        ChordFactor=WingChordFactor,
                                        max_degree=3)

# Evaluate the root chord:
RootChord = Wing.RootChord

# Here the TOPO_DS Shape to be displayed is stored in the Lifting surface 'Shape' Attribute
# from OCC.Graphic3d import Graphic3d_NOM_ALUMINIUM
# display.DisplayShape(Wing.Shape, update=True, material=Graphic3d_NOM_ALUMINIUM)


# # from OCC.AIS import AIS_WireFrame
# # display.Context.SetDisplayMode(AIS_WireFrame)
# # display.Repaint()
# display.DisplayShape(W2, update=True)


## Tailplane, Transonic Airliner

In [ ]:
from OCC.gp import gp_Ax1, gp_Pnt, gp_Dir
from airconics.examples.tailplane_example_transonic_airliner import *

# Position of the apex of the fin
P = [36.98-0.49-0.02, 0.0, 2.395-0.141]

SegmentNo = 100

Fin = liftingsurface.LiftingSurface(P, mySweepAngleFunctionFin,
                                    myDihedralFunctionFin,
                                    myTwistFunctionFin,
                                    myChordFunctionFin,
                                    myAirfoilFunctionFin,
                                    SegmentNo=SegmentNo,
                                    ChordFactor=FinChordFact,
                                    ScaleFactor=FinScaleFact)

#    Create the rotation axis centered at the apex point in the x direction
RotAxis = gp_Ax1(gp_Pnt(*P), gp_Dir(1, 0, 0))


Fin.Rotate(RotAxis, 90)
# display.DisplayShape(Fin.Shape, update=True)

# Position of the apex of the tailplane
P = [43, 0.000, 1.633+0.02]

SegmentNo = 100
ChordFactor = 1.01
ScaleFactor = 17.3

TP = liftingsurface.LiftingSurface(P, mySweepAngleFunctionTP,
                                   myDihedralFunctionTP,
                                   myTwistFunctionTP,
                                   myChordFunctionTP,
                                   myAirfoilFunctionTP,
                                   SegmentNo=SegmentNo,
                                   ChordFactor=TPChordFact,
                                   ScaleFactor=TPScaleFact)

# display.DisplayShape(TP.Shape, update=True)


## Fuselage Transonic Airliner

In [ ]:
from airconics.fuselage_oml import Fuselage

Fus = Fuselage(NoseLengthRatio, TailLengthRatio, Scaling=FuselageScaling,
             NoseCoordinates=[0., 0., 0],
             CylindricalMidSection=False,
             Maxi_attempt=5)

# from OCC.Graphic3d import Graphic3d_NOM_ALUMINIUM
# display.DisplayShape(Fus.OMLSurf, update=True, material=Graphic3d_NOM_ALUMINIUM)

act.export_STEPFile([Fus.OMLSurf], 'fuselage.stp')


## Wing-Body Fairing:

In [ ]:
# WingBodyFairing:
WTBFZ = RootChord*0.009 #787: 0.2
WTBFheight = 1.8*0.1212*RootChord #787:2.7
WTBFwidth = 1.08*FuselageWidth
WTBFXCentre = WingApex[0] + RootChord/2.0 + RootChord*0.1297 # 787: 23.8
WTBFlength = 1.167*RootChord #787:26

WBF = act.make_ellipsoid([WTBFXCentre, 0, WTBFZ], WTBFlength, WTBFwidth, WTBFheight)


## Miscelaneous operations

In [ ]:
# Trim the inboard section of the main wing:
CutCirc = act.make_circle3pt([0,WTBFwidth/4.,-45], [0,WTBFwidth/4.,45], [90,WTBFwidth/4.,0])
CutCircDisk = act.PlanarSurf(CutCirc)
Wing.Shape = act.TrimShapebyPlane(Wing.Shape, CutCircDisk)

#Mirror the main wing and tailplane:
Wing2 = act.mirror(Wing.Shape, plane='xz')
TP2 = act.mirror(TP.Shape, plane='xz', copy=True)


## Ipython Cell Renderer:

In [ ]:
# Note : This seems to make objects render in ipython notebook (only works for shapes?)

from airconics.Addons.WebServer import TornadoWeb
renderer = TornadoWeb.TornadoWebRenderer()
#    display all entities:
# Fuselage and wing-body fairing
renderer.DisplayShape(Fus.OMLSurf)
renderer.DisplayShape(WBF)

# #The Wings:
renderer.DisplayShape(Wing.Shape)
renderer.DisplayShape(Wing2)

#The Tailplane:
renderer.DisplayShape(TP.Shape)
renderer.DisplayShape(TP2)

#The Fin:
renderer.DisplayShape(Fin.Shape)
renderer

## Random things I tried out (Needs removing)

In [ ]:
# from OCC.gp import gp_Lin
# from OCC.ShapeConstruct import shapeconstruct

# Af2 = airconics.primitives.Airfoil([0.,0.,0.], ChordLength=3., Naca4Profile='2412')

# h = Af2.curve.GetHandle()
# EP = h.EndPoint()
# SP = h.StartPoint()
# closure = gp_Lin(EP, SP)

# shapeconstruct.JoinCurves(h.GetObject(), closure.)

# display.DisplayShape(Af2.Curve, update=True)

In [ ]:
# from OCC.gp import gp_Lin, gp_Ax1, gp_Dir
# from OCC.ShapeConstruct import shapeconstruct
# from OCC.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

# Af2 = airconics.primitives.Airfoil([0.,0.,0.], ChordLength=3., Naca4Profile='2412')

# h = Af2.Curve.GetObject()
# EP = h.EndPoint()
# SP = h.StartPoint()
# closure = BRepBuilderAPI_MakeEdge(gp_Lin(gp_Ax1(EP, gp_Dir(SP.XYZ()))) ).Edge()

# display.DisplayShape(closure, update=True)
# display.DisplayShape(Af2.Curve, update=True)

In [ ]:
# import airconics
# import airconics.AirCONICStools as act
# %gui qt4
# # Initialise the display
# from OCC.Display.SimpleGui import init_display
# display, start_display, add_menu, add_function_to_menu = init_display()

# from OCC.gp import gp_Pnt
# c = act.points_to_BezierCurve([gp_Pnt(0, 1, 0), gp_Pnt(2, 5, 4)])
# h = c.Copy().GetObject()
# display.DisplayShape(h.GetObject(), update=True)

In [ ]:
# # This cell can be used to start a freecad window, create a new document and transfer the airconics
# # Wing shape to the viewer
# import sys
# sys.path.append('/usr/lib/freecad/lib')
# import FreeCAD
# import FreeCADGui
# %gui qt
# FreeCADGui.showMainWindow()

# doc = FreeCAD.newDocument()

# import Part

# import airconics 
# # Import all example functional definitions for the Common Research Model (CRM) Wing:
# from airconics.examples.wing_example_transonic_airliner import *

# # Position of the apex of the wing
# P = [0., 0., 0.]

# # Class definition
# NSeg = 10
# ChordFactor = 1
# ScaleFactor = 50

# Wing = liftingsurface.LiftingSurface(P, mySweepAngleFunctionAirliner, 
#     myDihedralFunctionAirliner, 
#     myTwistFunctionAirliner, 
#     myChordFunctionAirliner, 
#     myAirfoilFunctionAirliner, SegmentNo=NSeg, ScaleFactor=ScaleFactor)

# w = Part.__fromPythonOCC__(Wing.Shape)
# Part.show(w)

In [ ]:
# shape = Wing.Shape
# from OCC.TopExp import TopExp_Explorer
# from OCC.TopAbs import TopAbs_FACE
# from OCC.TopoDS import TopoDS_Face
# ex = TopExp_Explorer(shape, TopAbs_FACE)
# ex.More()
# currentFace = ex.Current()